In [52]:
b = 10
num_blocks = 200
n = num_blocks*(b-1) + 1

A = spzeros(n, n)

for i = 1:num_blocks
    top_left = (i-1)*(b-1) + 1
    A[top_left:top_left+b-1, top_left:top_left+b-1] .= randn(b, b)
end

A = (A + A')/2;

B = speye(n, n);

In [66]:
nnz(Symmetric(A))

LoadError: [91mMethodError: no method matching nnz(::Symmetric{Float64,SparseMatrixCSC{Float64,Int64}})[0m
Closest candidates are:
  nnz([91m::SparseMatrixCSC[39m) at sparse/sparsematrix.jl:53
  nnz([91m::SparseVector[39m) at sparse/sparsevector.jl:37
  nnz([91m::Base.SparseArrays.UMFPACK.UmfpackLU[39m) at sparse/umfpack.jl:380
  ...[39m

In [83]:
@time A_copy = copy(A)

  0.000071 seconds (10 allocations: 324.109 KiB)


1801×1801 SparseMatrixCSC{Float64,Int64} with 19801 stored entries:
  [1   ,    1]  =  -0.0533007
  [2   ,    1]  =  -0.289308
  [3   ,    1]  =  -0.0217574
  [4   ,    1]  =  0.73068
  [5   ,    1]  =  1.79176
  [6   ,    1]  =  0.891551
  [7   ,    1]  =  -0.596496
  [8   ,    1]  =  0.516255
  [9   ,    1]  =  1.49738
  [10  ,    1]  =  -0.731675
  ⋮
  [1801, 1800]  =  1.01978
  [1792, 1801]  =  0.0628454
  [1793, 1801]  =  -0.690835
  [1794, 1801]  =  0.0164524
  [1795, 1801]  =  0.130147
  [1796, 1801]  =  -0.768762
  [1797, 1801]  =  -0.233444
  [1798, 1801]  =  -0.377756
  [1799, 1801]  =  0.898424
  [1800, 1801]  =  1.01978
  [1801, 1801]  =  -0.0520544

In [86]:
Lchol = cholfact(A + 100*speye(size(A,1)))

Base.SparseArrays.CHOLMOD.Factor{Float64}
type:          LLt
method: simplicial
maxnnz:      10801
nnz:         10801


In [74]:
@time eigs(A, nev=1, which=:SM)

  0.013117 seconds (639 allocations: 5.910 MiB, 57.78% gc time)


([-0.00185273], [-3.96859e-16; 2.97255e-16; … ; 5.76689e-21; 1.18549e-21], 1, 1, 20, [-0.799486, 0.652292, 0.755771, 0.54035, -0.444673, -0.254498, 0.788701, 0.290024, -0.00807781, -1.09429  …  0.317311, -0.303922, 0.763047, -3.36372, 1.63957, 0.588509, -0.606875, 1.32424, 0.326494, 0.0671169])

In [57]:
# grad(chol_L, B) = -1 - trace(chol_L \ B)
@time begin

function grad(chol_L::SparseArrays.CHOLMOD.Factor{Float64}, B::SparseMatrixCSC{Float64, Int64}, nu::Float64)
    tr = 0
    e_i = zeros(size(B, 2))
    e_i[1] = 1
    for i = 1:size(B, 2)
        if i > 1
            e_i[i-1] = 0
            e_i[i] = 1
        end
        tr += (chol_L \ e_i)[i]
    end
    return -1 + nu*tr
end
    
eval_f(chol_L::SparseArrays.CHOLMOD.Factor, t::Float64, nu::Float64) = -t - nu*logdet(chol_L)

curr_t = -100.
step_size = 10
nu = 1e-3

L = A - curr_t*B
init_chol_L = cholfact(L)
chol_L = copy(init_chol_L)

converged = false
    
prev_val = Inf
    
for i = 1:100
    curr_grad = grad(chol_L, B, nu)
    is_pos_def = false
        
    while !is_pos_def
        tent_t = curr_t - step_size*curr_grad
        L .= A - tent_t*B

        try
            cholfact!(chol_L, L)
            
            curr_eval = eval_f(chol_L, tent_t, nu)
            
            if curr_eval > prev_val
                step_size *= .5
                continue
            end
            
            prev_val = curr_eval
            println("curr t : $tent_t")
            is_pos_def = true
                
            if abs(curr_grad)^2 < 1e-4
                if nu < 1e-4
                    converged = true
                    break
                end
                
                nu *= .1
                prev_val = Inf
                break
            end
                
            curr_t = tent_t
            step_size *= 1.2
            break

        catch y
            if isa(y, Base.LinAlg.PosDefException)
                chol_L = copy(init_chol_L)
                step_size *= .5
            else
                error("wtf?")
            end
        end
    end
    
    if converged
        break
    end
end
    
end

curr t : -90.18020967812741
curr t : -78.42004221830292
curr t : -64.35107958405894
curr t : -47.555402829058394
curr t : -27.606805205825275
curr t : -15.98333885147588
curr t : -9.380243121667235
curr t : -5.83057124295639
curr t : -5.734985172880681
curr t : -5.7085878115094895
curr t : -5.694443697684
curr t : -5.695138745308263
curr t : -5.694810620903282
curr t : -5.694946042648024
curr t : -5.694883680873104
curr t : -5.694885226009103
curr t : -5.693825754496518
curr t : -5.693235178769239
curr t : -5.69298078531797
curr t : -5.693124378944201
curr t : -5.693095508634895
curr t : -5.693071221235116
curr t : -5.693057104002219
curr t : -5.69305439203153
curr t : -5.693054944822666
curr t : -5.692983304353891
curr t : -5.692966771066924
curr t : -5.69295510135826
curr t : -5.692962309619153
curr t : -5.692960220293733
curr t : -5.692959636529371
curr t : -5.692959753825523
  2.694919 seconds (825.90 k allocations: 5.510 GiB, 8.59% gc time)


In [18]:
eigs(A,nev=1,which=:SR)

([-6.17813], [-8.86227e-19; 1.70165e-18; … ; -7.09836e-17; 1.31684e-18], 1, 9, 100, [-0.00832226, -0.00582737, -0.0372743, 0.0239053, 0.0278231, 0.00543517, 0.0221169, -0.00404483, -0.0160135, -0.00235164  …  0.047227, -0.0434156, -0.00583877, 0.0929452, -0.054085, 0.0507873, -0.0536809, -0.0842467, -0.109428, 0.07352])

In [14]:
nnz(chol_L\B)

2940308

In [74]:
func(x, t, nu) = -t - nu*log(x - t)
grad(x, t, nu) = -1 + nu/(x - t)

curr_t = -100
step_size = 1

x_set = 1
nu = 1
epsilon = 1e-6
prev_val = Inf

for i=1:100
    curr_grad = grad(x_set, curr_t, nu)
    tent_t = curr_t - step_size*curr_grad
    
    if tent_t > x_set
        step_size *= .5
        continue
    end
    
    tent_val = func(x_set, tent_t, nu)
    
    if tent_val > prev_val
        step_size *= .5
        continue
    end
    
    prev_val = tent_val
    
    if abs(curr_grad)^2 < epsilon
        println("Finished optimizing iteration at nu : $nu with t $tent_val")
        if nu < 1e-4
            break
        end
        prev_val = Inf
        nu *= .1
    end
    
    curr_t = tent_t
    
    step_size *= 1.2
end

Finished optimizing iteration at nu : 1 with t 1.0484698606752908e-9
Finished optimizing iteration at nu : 0.1 with t -0.669741490289786
Finished optimizing iteration at nu : 0.010000000000000002 with t -0.9439482969657267
Finished optimizing iteration at nu : 0.0010000000000000002 with t -0.9920922447141156
Finished optimizing iteration at nu : 0.00010000000000000003 with t -0.9989789659628019
Finished optimizing iteration at nu : 1.0000000000000004e-5 with t -0.9998748707422735
